In [39]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [40]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [41]:
# wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [42]:
wiki_tool.run("HUNTER X HUNTER")

'Page: Hunter × Hunter\nSummary: Hunter × Hunter (pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga m'

In [43]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4

loader = WebBaseLoader("https://www.ign.com/articles/destiny-2s-year-of-prophecy-heres-everything-guardians-need-to-know", bs_kwargs=dict(parse_only=bs4.SoupStrainer(["p", "span", "h1", "h2", "h3", "i"])))
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama2"))
retriever = vectorDb.as_retriever()


In [44]:
from langchain.tools.retriever import create_retriever_tool ## It is a tool that allows you to create a retriever tool from a retriever object.

retriever_tool = create_retriever_tool(retriever=retriever, name="Destiny2YearOfProphecyRetriever", description="A tool to retrieve information about Destiny 2's Year of Prophecy from a vector database. It can answer questions about the game, its lore, and related topics. Use it to get detailed information and insights about the game.")

In [45]:
retriever_tool.run("What is Destiny 2's Year of Prophecy?")

'was only in the last month that we got our first real look at it. In IGN\'s Marathon hands-on preview, we wrote: "After playing enough Marathon to get a pretty good feel for it in its alpha state, I’m already fairly confident it’ll scratch the Bungie PvP itch that’s been left unscratched for a number of years now. Here’s hoping they’ll knock it out of the park when it comes to PC and consoles in September."Vikki Blake is a reporter, critic, columnist, and consultant. She\'s also a Guardian, Spartan, Silent Hillian, Legend, and perpetually High Chaos. Find her at BlueSky.In This ArticleOverviewThe Final ShapeLightfallThe Witch Queen••••••••••••••••••••\n\navailable to all players for no additional cost. While the dungeons themselves — Prophecy, Spire of the Watcher, and Ghosts of the Deep — will be familiar to some, even the most hardened Guardian can expect "unique twists" in which you\'ll reconnect with Orin and refreshed dungeon weapons.Then, on July 15, we\'ll get the brand-new exp

In [46]:
##Arxiv is a tool that allows you to search for academic papers on the Arxiv preprint server. It can be used to find papers on a wide range of topics, including computer science, physics, mathematics, and more. 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [47]:
tools = [wiki_tool, arxiv_tool, retriever_tool]

In [48]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Destiny2YearOfProphecyRetriever', description="A tool to retrieve information about Destiny 2's Year of Prophecy from a vector database. It can answer questions about the game, its lore, and related topics. Use it to get detailed information and insights about the game.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_document

In [58]:
from dotenv import load_dotenv
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_ollama import OllamaLLM

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

llm1 = GoogleGenerativeAI(model="gemini-2.0-flash-exp")
llm2 = OllamaLLM(model="llama2")

In [82]:
from langchain import hub

prompt1 = hub.pull("deanmachines-ai/superb_system_instruction_prompt")
prompt1.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="# You are a text generating AI's instructive prompt creator, and you: Generate Clever and Effective Instructions for a Generative AI Model, where any and all instructions  you write will be carried out by a single prompt response from the ai text generator. Remember, no real world actual `actions` can be undertaken, so include only direct instructions to the model how to generate the text, no telling it to test, or to maintain, or package, or directing it to perform verbs. no verbs..\n\n1. Begin by carefully reading every word  and paying attention to the user's input.  \n What are they needing a set of instructions to be written for. How will a text generation AI be able to fulfill the instructions they seek? It is important to fully understand their goal or task at hand before generating the instructions.\n\n2. Analyze the user's input to identify the specific types 

In [65]:
prompt2 = hub.pull("klcoder/mistral-functioncalling")
prompt2

PromptTemplate(input_variables=['question', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'klcoder', 'lc_hub_repo': 'mistral-functioncalling', 'lc_hub_commit_hash': '31566a970d06280aefb3bce3b48b707dff1f3feb89fbe8dddc2c2c95b66d4a9f'}, template="SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{tools}\n\nThe output needs to be in the following format:\n{{\n    'name': <function name>,\n    'arguments': <arguments to pass to the function>\n}}\n\nFor questions not related to the tools above, respond with an empty json object.\n\nUser: {question}\nFUNCTION: ")

In [83]:
## Agents: The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order. 

from langchain.agents import initialize_agent, AgentType

gemini_agent = initialize_agent(
    tools=tools,
    llm=llm1,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # agent_kwargs={"prompt":prompt1}
)

In [67]:
ollama_agent = initialize_agent(
    tools=tools,
    llm=llm2,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prompt":prompt2}
)

In [84]:
## Agent Executor

from langchain.agents import AgentExecutor
agentExecutor1 = AgentExecutor(agent=gemini_agent, tools=tools, verbose=True)
agentExecutor1

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=False, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nDestiny2YearOfProphecyRetriever(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetri

In [ ]:
print(agentExecutor1.input_keys)


[]


In [68]:
agentExecutor2 = AgentExecutor(agent=ollama_agent, tools=tools, verbose=True)
agentExecutor2

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=False, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nDestiny2YearOfProphecyRetriever(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetri

In [71]:
print(agentExecutor2.input_keys)

[]


In [86]:
agentExecutor1.invoke({"input":"What is Destiny 2's Year of Prophecy?"})



> Entering new AgentExecutor chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'

In [74]:
agentExecutor2.invoke({"question":"What is Destiny 2's Year of Prophecy?"})



> Entering new AgentExecutor chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'